# Merge y Join en Pandas

Este notebook demuestra cómo combinar múltiples DataFrames usando merge y join, similar a SQL JOINs.

**Referencia:** [Merge y Join](../pandas/python-para-datos/05-merge-join.md)

**Objetivos:**
- Realizar diferentes tipos de merge (inner, left, right, outer)
- Combinar DataFrames con diferentes nombres de columnas
- Manejar valores nulos después de merge
- Verificar integridad de datos antes de mergear
- Usar índices para joins eficientes

## 1. Importar librerías y cargar datos

In [ ]:
import pandas as pd
import numpy as np

# Cargar datos desde los CSVs de ejemplo
ventas = pd.read_csv('../data/ventas.csv')
productos = pd.read_csv('../data/productos.csv')

print(f"✅ Datos cargados")
print(f"\nVentas: {len(ventas)} registros")
print(f"Productos: {len(productos)} registros")
print(f"\nColumnas en ventas: {ventas.columns.tolist()}")
print(f"Columnas en productos: {productos.columns.tolist()}")

print("\n=== PRIMERAS FILAS DE VENTAS ===")
print(ventas.head())

print("\n=== PRIMERAS FILAS DE PRODUCTOS ===")
print(productos.head())

## 2. Verificar datos antes de mergear

In [ ]:
print("=== VERIFICACIÓN DE DUPLICADOS ===")
print(f"Duplicados en 'producto' (ventas): {ventas['producto'].duplicated().sum()}")
print(f"Duplicados en 'producto' (productos): {productos['producto'].duplicated().sum()}")

print("\n=== VERIFICACIÓN DE COINCIDENCIAS ===")
productos_en_ventas = ventas['producto'].isin(productos['producto']).sum()
print(f"Productos de ventas que están en catálogo: {productos_en_ventas} de {len(ventas)}")
print(f"Productos en catálogo que tienen ventas: {productos['producto'].isin(ventas['producto']).sum()} de {len(productos)}")

print("\n=== PRODUCTOS ÚNICOS ===")
print(f"Productos únicos en ventas: {ventas['producto'].nunique()}")
print(f"Productos únicos en catálogo: {productos['producto'].nunique()}")

# Verificar si hay productos en ventas que no están en catálogo
productos_solo_ventas = set(ventas['producto']) - set(productos['producto'])
productos_solo_catalogo = set(productos['producto']) - set(ventas['producto'])

if productos_solo_ventas:
    print(f"\n⚠️ Productos en ventas pero no en catálogo: {productos_solo_ventas}")
else:
    print("\n✅ Todos los productos de ventas están en el catálogo")

if productos_solo_catalogo:
    print(f"ℹ️ Productos en catálogo sin ventas: {len(productos_solo_catalogo)}")
else:
    print("✅ Todos los productos del catálogo tienen ventas")

## 3. Tipos de merge

### 3.1 Inner Join

In [ ]:
# Inner join: Solo filas que coinciden en ambas tablas
df_inner = pd.merge(ventas, productos, on='producto', how='inner')

print("=== INNER JOIN ===")
print(f"Registros en ventas: {len(ventas)}")
print(f"Registros en productos: {len(productos)}")
print(f"Registros después de inner join: {len(df_inner)}")
print("\nPrimeras filas:")
print(df_inner[['producto', 'total', 'stock', 'proveedor']].head())

### 3.2 Left Join

In [ ]:
# Left join: Todas las filas de la izquierda (ventas)
df_left = pd.merge(ventas, productos, on='producto', how='left')

print("=== LEFT JOIN ===")
print(f"Registros después de left join: {len(df_left)}")
print(f"Ventas sin producto en catálogo: {df_left['stock'].isnull().sum()}")

print("\nPrimeras filas:")
print(df_left[['producto', 'total', 'stock', 'proveedor']].head())

# Mostrar ventas sin información de producto (si las hay)
if df_left['stock'].isnull().sum() > 0:
    print("\n⚠️ Ventas sin información de producto:")
    print(df_left[df_left['stock'].isnull()][['producto', 'total']])

### 3.3 Right Join

In [ ]:
# Right join: Todas las filas de la derecha (productos)
df_right = pd.merge(ventas, productos, on='producto', how='right')

print("=== RIGHT JOIN ===")
print(f"Registros después de right join: {len(df_right)}")
print(f"Productos sin ventas: {df_right['id'].isnull().sum()}")

print("\nPrimeras filas:")
print(df_right[['producto', 'total', 'stock', 'proveedor']].head())

# Mostrar productos sin ventas (si los hay)
if df_right['id'].isnull().sum() > 0:
    print("\nℹ️ Productos sin ventas:")
    print(df_right[df_right['id'].isnull()][['producto', 'stock', 'proveedor']])

### 3.4 Outer Join

In [ ]:
# Outer join: Todas las filas de ambas tablas
df_outer = pd.merge(ventas, productos, on='producto', how='outer')

print("=== OUTER JOIN ===")
print(f"Registros después de outer join: {len(df_outer)}")

ventas_sin_producto = (df_outer['id'].notna() & df_outer['stock'].isna()).sum()
productos_sin_ventas = (df_outer['id'].isna() & df_outer['stock'].notna()).sum()

print(f"Ventas sin producto en catálogo: {ventas_sin_producto}")
print(f"Productos sin ventas: {productos_sin_ventas}")

print("\nPrimeras filas:")
print(df_outer[['producto', 'total', 'stock', 'proveedor']].head())

## 4. Ejemplos prácticos

### 4.1 Enriquecer ventas con información de productos

In [ ]:
# Merge para enriquecer ventas con información de productos
ventas_completas = pd.merge(
    ventas, 
    productos[['producto', 'stock', 'proveedor', 'fecha_lanzamiento']], 
    on='producto', 
    how='left'
)

print("=== VENTAS CON INFORMACIÓN DE PRODUCTOS ===")
print(ventas_completas[['producto', 'total', 'stock', 'proveedor', 'fecha_lanzamiento']].head(10))

# Análisis: productos con bajo stock y altas ventas
productos_riesgo = ventas_completas[
    (ventas_completas['stock'] < 20) & 
    (ventas_completas['total'] > 500)
]

print("\n=== PRODUCTOS CON BAJO STOCK Y ALTAS VENTAS ===")
if len(productos_riesgo) > 0:
    print(productos_riesgo[['producto', 'total', 'stock', 'proveedor']])
else:
    print("No hay productos con bajo stock y altas ventas simultáneamente")

### 4.2 Merge con sufijos para columnas duplicadas

In [ ]:
# Ambas tablas tienen 'precio' y 'categoria', usamos sufijos
df_sufijos = pd.merge(
    ventas, 
    productos, 
    on='producto', 
    suffixes=('_venta', '_base'),  # precio_venta vs precio_base
    how='inner'
)

print("=== MERGE CON SUFIJOS ===")
print("Columnas con sufijos:")
print([col for col in df_sufijos.columns if 'precio' in col or 'categoria' in col])

print("\nComparación de precios:")
df_sufijos['diferencia_precio'] = df_sufijos['precio_venta'] - df_sufijos['precio_base']
print(df_sufijos[['producto', 'precio_venta', 'precio_base', 'diferencia_precio']].head(10))

### 4.3 Análisis de ventas por proveedor

In [ ]:
# Combinar ventas con productos para analizar por proveedor
ventas_proveedor = pd.merge(ventas, productos[['producto', 'proveedor']], on='producto', how='left')

print("=== VENTAS POR PROVEEDOR ===")
ventas_por_proveedor = ventas_proveedor.groupby('proveedor').agg({
    'total': 'sum',
    'id': 'count',
    'cantidad': 'sum'
}).round(2)
ventas_por_proveedor.columns = ['Total_Ventas', 'Num_Transacciones', 'Cantidad_Total']
ventas_por_proveedor = ventas_por_proveedor.sort_values('Total_Ventas', ascending=False)
print(ventas_por_proveedor)

print(f"\n🏆 Proveedor con mayores ingresos: {ventas_por_proveedor.index[0]}")
print(f"   Total: €{ventas_por_proveedor.iloc[0]['Total_Ventas']:,.2f}")

## 5. Usar índices para joins eficientes

In [ ]:
# Si las claves son índices (más eficiente para merges repetidos)
ventas_indexed = ventas.set_index('producto')
productos_indexed = productos.set_index('producto')

# Usar join() en lugar de merge()
df_joined = ventas_indexed.join(productos_indexed[['stock', 'proveedor']], how='left')

print("=== JOIN USANDO ÍNDICES ===")
print(f"Registros después de join: {len(df_joined)}")
print("\nPrimeras filas:")
print(df_joined[['total', 'stock', 'proveedor']].head())

# Resetear índice si necesitas 'producto' como columna
df_joined_reset = df_joined.reset_index()
print("\n✅ Índice reseteado, 'producto' es columna nuevamente")

## 6. Análisis completo: Ventas enriquecidas con productos

Este ejemplo combina todas las técnicas aprendidas.

In [ ]:
# 1. Combinar ventas con productos
ventas_completas = pd.merge(ventas, productos, on='producto', how='left', suffixes=('_venta', '_base'))

# 2. Crear resumen de ventas por producto
resumen_ventas = ventas.groupby('producto').agg({
    'total': 'sum',
    'cantidad': 'sum',
    'id': 'count'
}).reset_index()
resumen_ventas.columns = ['producto', 'total_ventas', 'cantidad_total', 'num_transacciones']

# 3. Combinar resumen con información de productos
analisis_completo = pd.merge(resumen_ventas, productos, on='producto', how='left')

# 4. Calcular métricas adicionales
analisis_completo['ratio_ventas_stock'] = analisis_completo['total_ventas'] / analisis_completo['stock']
analisis_completo['ventas_por_transaccion'] = analisis_completo['total_ventas'] / analisis_completo['num_transacciones']

# 5. Ordenar por ratio de ventas vs stock
analisis_completo = analisis_completo.sort_values('ratio_ventas_stock', ascending=False)

print("=" * 70)
print("ANÁLISIS COMPLETO: VENTAS ENRIQUECIDAS CON PRODUCTOS")
print("=" * 70)
print("\nTop 10 productos con mayor ratio ventas/stock (más demandados):")
print(analisis_completo[['producto', 'total_ventas', 'stock', 'proveedor', 'ratio_ventas_stock']].head(10))

print("\n📊 Resumen por proveedor:")
resumen_proveedor = analisis_completo.groupby('proveedor').agg({
    'total_ventas': 'sum',
    'stock': 'sum',
    'num_transacciones': 'sum'
}).round(2)
resumen_proveedor.columns = ['Total_Ventas', 'Stock_Total', 'Transacciones']
print(resumen_proveedor.sort_values('Total_Ventas', ascending=False))

print("\n" + "=" * 70)